#  Instruction

In this notebook, we will learn how to implement DQN using Tensorflow for the [Cartpole environment in OpenAI gym](https://gymnasium.farama.org/environments/classic_control/cart_pole/). You are given a basic skeleton but you need to complete the code where appropriate to solve the cartpole problem.

You are free to tweak the code at any part. Your are also free to tweak the hyper-parameters to improve the performance of the agent. At the end you have to evaluate the performance of the agent on 100 independent episodes on the environment and print out the average testing performance.

Make sure that your final submission is a notebook that can be run from beginning to end, and that you print out the performance of the agent at the end of the notebook.

In [ ]:
import os
!{os.sys.executable} -m pip install gymnasium
!{os.sys.executable} -m pip install Pillow
!{os.sys.executable} -m pip install ipython
!{os.sys.executable} -m pip install pygame

from PIL import Image
from IPython.display import display

import tensorflow as tf
from collections import deque
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import gym
from tensorflow import keras
from tensorflow.keras import layers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Parameters
gamma = 0.95  # discount 
envname = "CartPole-v1"  # environment name
env=gym.make(envname, render_mode="rgb_array")

obssize = env.observation_space.low.size
actsize = env.action_space.n

/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


## DQN (Deep Q Network)

In previous HWs, we have learned to use Tensorflow to build deep learning models. In this HW, we will apply deep learning as function approximations in reinforcement learning. 

Reference: DQN https://arxiv.org/abs/1312.5602


In tabular Q-learning, we maintain a table of state-action pairs $(s,a)$ and save one action value for each entry $Q(s,a),\forall (s,a)$. At each time step $t$, we are in state $s_t$, then we choose action based on $\epsilon-$greedy strategy. With prob $\epsilon$, choose action uniformly random; with prob $1-\epsilon$, choose action based on $$a_t = \arg\max_a Q(s_t,a)$$ 

We then get the instant reward $r_t$, update the Q-table using the following rule

$$Q(s_t,a_t) \leftarrow (1-\alpha)Q(s_t,a_t) + \alpha (r_t + \max_a \gamma Q(s_{t+1},a))$$

where $\alpha \in (0,1)$ is learning rate. The algorithm is shown to converge in tabular cases. However, in cases where we cannot keep a table for state and action, we need function approximation. Consider using neural network with parameter $\theta$, the network takes as input state $s$ and action $a$. (*or some features of state and action*). Let $Q_\theta(s,a)$ be the output of the network, which estimates the optimal Q-value function for state $s$ and action $a$.
$$Q_\theta(s,a) \approx Q^\ast(s,a)$$

In [34]:
def model_creator():
    model = keras.Sequential()
    model.add(layers.Dense(24,activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal))

    model.add(layers.Dense(24,activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal))
    # you should later modify this neural network
    model.add(layers.Dense(actsize, kernel_initializer=tf.keras.initializers.GlorotNormal)) # you should have one output for each possible action
    return model

We wish to train the neural network in order to find $\theta$ such that $Q_\theta(s,a)$ approximates $Q^*(s,a)$. As we discussed in the class, we can use observations of form $(s_i, a_i, r_i, s'_{i})$ (i.e., observing reward $r_i$ and new state $s'_{i}$ on taking action $a_i$ in state $s_i$) for training. Based on observations, we can use stochastic gradient descent to update $\theta$ in the direction that minimizes the loss function. Further, based on values $Q_\theta(s,a)$, we can choose the action based on $\epsilon$-greedy policy.

Formally let $d_i$ be the target for the $i$-th sample $(s_t,a_t,r_t,s_{t+1})$

$$d_i =  r_t +   \gamma \max_a Q_\theta(s_{t+1},a)$$

We can collect a batch of $N$ samples (this generalizes the per sample update $N=1$ discusssed in class), consider the loss fucntion,

$$J:=\frac{1}{N} \sum_{i=1}^N (Q_\theta(s_i,a_i) - d_i)^2$$

and update

$$
\theta \leftarrow \theta -\alpha \nabla_\theta J
$$

This procedure has been shown to be fairly unstable. In class, we discussed two techniques to stabilize the training process: target network and replay buffer.

**Replay Buffer**
Maintain a buffer $R$ to store trainsition tuples $(s_t,a_t,r_t,s_{t+1})$. When minimizing the loss, we sample batches from the replay buffer and compute gradients for update on these batches. In particular, in each update, we sample $N$ tuples $(s_t,a_t,r_t,s_{t+1})$ from buffer $R$ and then minimize the
loss 

$$\frac{1}{N} \sum_{i=1}^N (Q_\theta(s_i,a_i) -  (r_i + \gamma \max_a Q_\theta(s_i^\prime,a))^2$$

and update parameters.

**Target Network**
Maintain a target network in addition to the original principal network. The target network is just a copy of the original network but the parameters are not updated by gradients. The target network $\theta^{-}$ is copied from the principal network every $\tau$ time steps. Target network is used to compute the targets for update

$$d_i = \max_a r_t + \gamma Q_{\theta^{-}}(s_{i}^\prime,a)$$

the targets are used in the loss function to update the principal network parameters. This slowly updated target network ensures that the targets come from a relatively stationary distribution and hence stabilize learning.

In [37]:
batch_size = 24
# Model used for selecting actions (principal)
model = model_creator()
# Then create the target model. This will periodically be copied from the principal network 
model_target = model_creator()

model.build((batch_size,obssize,))
model_target.build((batch_size,obssize,))

- Complete the code below to learn an Agent using DQN. 
- You should tweak the Neural network appropriately to achieve a good reward (>100). Ideally you would want to have a reward close to 200.
- The reference paper performs updates every 4 actions. You can experiment with this parameter to speed up the learning
- You can experiment with other parameters as well, like learning rate, memory size, different exploration schemes (e.g., adaptive $\epsilon$ or strategic explorations with bonus rewards) and others.

- As we mentioned in class, there are multiple ways to improve the efficiency even further. OPTIONALLY you can experiment with these:
  - Prioritized Replay buffer.
  - Double DQN 
  -Dueling DQN architecture.

- In case you need to debug your code you can try printing relevant information as the training happens. For example although the performance might vary from iteration to iteration, the average Q values might increase overtime in a more smooth way. This is discussed in the refernece paper

- Create a plot of the running reward sampled throughout the training at the frequency of your choice at the end of the training
- OPTIONALLY you can create a plot for the average Q-values of the principal Q-network sampled at the frequency of your choice

- Ideally you want to learn with as few episodes as possible. However you will not be graded on sample efficiency in this homework. You encouraged to try to learn efficiently though.

- Note that the skeleton code includes the GradientTape construct to do the learning. Take a look [here](https://www.tensorflow.org/api_docs/python/tf/GradientTape) for an explanation of GradientTape. It allows for more flexibility than model.fit. Also it uses Adam (Adaptive Moment Estimation) for Stochastic Gradient Descent optimizer. 

In [38]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Our Experience Replay memory 
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []

# Replay memory size
max_memory = 2500 # You can experiment with different sizes.

running_reward = 0
episode_count = 0
timestep_count = 0


# how often to train your model - this allows you to speed up learning
# by not performing in every iteration learning. See also refernece paper
# you can set this value to other values like 1 as well to learn every time 

update_after_actions = 25

epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.999

# How often to update the target network
target_update_every = 1000
#loss_function = keras.losses.Huber() # You can use the Huber loss function or the mean squared error function 
loss_function = keras.losses.MeanSquaredError()
max_steps_per_episode = 1000

max_episodes = 2500
# max_steps_per_episode = 1000
last_n_reward = 100

for episode in range(max_episodes):
    state = np.array(env.reset())
    episode_reward = 0

    epsilon = max(epsilon*epsilon_decay, epsilon_min)

    for timestep in range(1, max_steps_per_episode):
        timestep_count += 1

        # exploration
        if np.random.rand() < epsilon:
            # Take random action
            action = np.random.choice(actsize)
        else:
            # Predict action Q-values
            # From environment state
            state_t = tf.convert_to_tensor(state)
            state_t = tf.expand_dims(state_t, 0)
            action_vals = model(state_t, training=False)
            # Choose the best action
            action = np.argmax(action_vals[0])

        # follow selected action
        state_next, reward, done, _ = env.step(action)
        #state_next = np.array(state_next)
        episode_reward += reward

        # Save action/states and other information in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        rewards_history.append(reward)
        done_history.append(done)

        state = state_next

        # Update every Xth frame to speed up (optional)
        # and if you have sufficient history
        if timestep_count % update_after_actions == 0 and len(action_history) > batch_size:

            #  Sample a set of batch_size memories from the history

            indexes = np.random.randint(low=0, high=len(action_history) - 1, size=batch_size)
            state_sample = tf.convert_to_tensor([state_history[x] for x in indexes])
            state_next_sample = tf.convert_to_tensor([state_next_history[x] for x in indexes])
            rewards_sample = [rewards_history[x] for x in indexes]
            action_sample = [action_history[x] for x in indexes]
            done_sample = tf.cast(tf.convert_to_tensor([done_history[x] for x in indexes]), tf.float32)


            # Create for the sample states the targets (r+gamma * max Q(...) )
            Q_next_state = tf.math.reduce_max(model_target(state_next_sample, training=False), axis=1)

            # If the episode was ended (done_sample value is 1)
            # you can penalize the Q value of the target by some value `penalty`
            # Q_targets = Q_targets * (1 - done_sample) - penalty*done_sample
            Q_targets = rewards_sample + gamma * Q_next_state * (1 - done_sample)


            # What actions are relevant and need updating
            relevant_actions = tf.one_hot(action_sample, actsize)
            # we will use Gradient tape to do a custom gradient 
            # in the `with` environment we will record a set of operations
            # and then we will take gradients with respect to the trainable parameters
            # in the neural network
            with tf.GradientTape() as tape:
                # Train the model on your action selecting network
                q_values = model(state_sample, training=False)
                # We consider only the relevant actions
                Q_of_actions = tf.reduce_sum(tf.multiply(q_values, relevant_actions), axis=1)
                # Calculate loss between principal network and target network
                loss = loss_function(Q_targets, Q_of_actions)

            # Nudge the weights of the trainable variables towards 
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if timestep_count % target_update_every == 0:
            # update the the target network with new weights
            model_target.set_weights(model.get_weights())
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {}, epsilon {}"
            print(template.format(running_reward, episode_count, timestep_count, epsilon))

        # Don't let the memory grow beyond the limit
        if len(rewards_history) > max_memory:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]
        if done: break

    # reward of last 100
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > last_n_reward: del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)
    episode_count += 1

    # If you want to stop your training once you achieve the reward you want you can
    # have an if statement here. Alternatively you can stop after a fixed number
    # of episodes.

running reward: 22.64 at episode 42, frame count 1000, epsilon 0.9578907814534664
running reward: 22.35 at episode 89, frame count 2000, epsilon 0.9138900318559524
running reward: 22.27 at episode 134, frame count 3000, epsilon 0.8736568985103146
running reward: 20.03 at episode 189, frame count 4000, epsilon 0.8268805241487632
running reward: 18.34 at episode 242, frame count 5000, epsilon 0.784176167005256
running reward: 18.09 at episode 298, frame count 6000, epsilon 0.7414484806367364
running reward: 16.70 at episode 359, frame count 7000, epsilon 0.6975506718651011
running reward: 15.07 at episode 426, frame count 8000, epsilon 0.6523241732116479
running reward: 13.80 at episode 498, frame count 9000, epsilon 0.6069859307919768
running reward: 14.88 at episode 556, frame count 10000, epsilon 0.572765620160788
running reward: 17.58 at episode 615, frame count 11000, epsilon 0.539934088348504
running reward: 17.68 at episode 668, frame count 12000, epsilon 0.5120491189128954
runnin

/usr/local/lib/python3.9/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment CartPole-v1 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


running reward: 17.70 at episode 775, frame count 14000, epsilon 0.4600646486360102
running reward: 23.98 at episode 798, frame count 15000, epsilon 0.44959874735743227
running reward: 29.64 at episode 826, frame count 16000, epsilon 0.43717846703394564
running reward: 33.38 at episode 858, frame count 17000, epsilon 0.4234034438366067
running reward: 39.21 at episode 877, frame count 18000, epsilon 0.41543077175087
running reward: 34.95 at episode 917, frame count 19000, epsilon 0.39913351012122433
running reward: 34.69 at episode 944, frame count 20000, epsilon 0.38849584071717547
running reward: 36.11 at episode 966, frame count 21000, epsilon 0.380038079308654
running reward: 37.25 at episode 989, frame count 22000, epsilon 0.3713926834236692
running reward: 40.72 at episode 1015, frame count 23000, epsilon 0.36185621618376534
running reward: 42.83 at episode 1035, frame count 24000, epsilon 0.3546874337726758
running reward: 46.36 at episode 1053, frame count 25000, epsilon 0.3483

/usr/local/lib/python3.9/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment CartPole-v1 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


running reward: 98.61 at episode 1735, frame count 96000, epsilon 0.17607089032534334
running reward: 104.42 at episode 1743, frame count 97000, epsilon 0.17466724334001496
running reward: 112.40 at episode 1751, frame count 98000, epsilon 0.17327478630695967
running reward: 120.97 at episode 1757, frame count 99000, epsilon 0.1722377332480149
running reward: 130.10 at episode 1762, frame count 100000, epsilon 0.17137826523759098
running reward: 137.44 at episode 1769, frame count 101000, epsilon 0.17018221033225317
running reward: 144.65 at episode 1776, frame count 102000, epsilon 0.16899450273591984
running reward: 149.44 at episode 1783, frame count 103000, epsilon 0.16781508419242955
running reward: 152.11 at episode 1789, frame count 104000, epsilon 0.1668107075597524
running reward: 158.66 at episode 1796, frame count 105000, epsilon 0.16564652979915298
running reward: 158.74 at episode 1800, frame count 106000, epsilon 0.1649849368967147
running reward: 159.84 at episode 1804, 

Evaluate the performance of the agent on 100 episodes on the environment and print out the average testing performance. Alternatively you can make sure the code above terminates with 100 episodes where there is no exploration at all (epsilon=0).

In [45]:
# YOUR CODE Here
test_episodes = 100

avg_reward = 0

for episode in range(test_episodes):
    state = np.array(env.reset())

    episode_reward = 0

    for timestep in range(1, 1000):
        state = tf.convert_to_tensor(state)
        state = tf.expand_dims(state, 0)
        action_vals = model(state, training=False)
        action = np.argmax(action_vals[0])
        state_next, reward, done, _ = env.step(action)

        episode_reward += reward
        avg_reward += reward

        state = state_next

        if done:
            template = "running reward: {:.2f} at episode {}"
            print(template.format(episode_reward, episode))
            break
template = "average reward: {:.2f} after {} episodes"
print(template.format(avg_reward/test_episodes, test_episodes))

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment CartPole-v1 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mod

running reward: 224.00 at episode 0
running reward: 244.00 at episode 1
running reward: 239.00 at episode 2
running reward: 187.00 at episode 3
running reward: 196.00 at episode 4
running reward: 194.00 at episode 5


/usr/local/lib/python3.9/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment CartPole-v1 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


running reward: 209.00 at episode 6
running reward: 217.00 at episode 7
running reward: 206.00 at episode 8
running reward: 235.00 at episode 9
running reward: 213.00 at episode 10
running reward: 201.00 at episode 11
running reward: 252.00 at episode 12
running reward: 237.00 at episode 13
running reward: 235.00 at episode 14
running reward: 219.00 at episode 15
running reward: 254.00 at episode 16
running reward: 262.00 at episode 17
running reward: 209.00 at episode 18
running reward: 222.00 at episode 19
running reward: 241.00 at episode 20
running reward: 191.00 at episode 21
running reward: 224.00 at episode 22
running reward: 217.00 at episode 23
running reward: 215.00 at episode 24


/usr/local/lib/python3.9/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment CartPole-v1 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


running reward: 224.00 at episode 25
running reward: 252.00 at episode 26
running reward: 191.00 at episode 27
running reward: 226.00 at episode 28
running reward: 214.00 at episode 29
running reward: 231.00 at episode 30
running reward: 196.00 at episode 31
running reward: 205.00 at episode 32
running reward: 241.00 at episode 33
running reward: 209.00 at episode 34
running reward: 203.00 at episode 35
running reward: 183.00 at episode 36
running reward: 226.00 at episode 37
running reward: 198.00 at episode 38
running reward: 192.00 at episode 39
running reward: 200.00 at episode 40
running reward: 203.00 at episode 41
running reward: 200.00 at episode 42
running reward: 212.00 at episode 43
running reward: 229.00 at episode 44
running reward: 214.00 at episode 45
running reward: 220.00 at episode 46
running reward: 240.00 at episode 47
running reward: 212.00 at episode 48
running reward: 199.00 at episode 49
running reward: 239.00 at episode 50
running reward: 225.00 at episode 51
r

/usr/local/lib/python3.9/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment CartPole-v1 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


running reward: 200.00 at episode 62
running reward: 231.00 at episode 63
running reward: 241.00 at episode 64
running reward: 226.00 at episode 65
running reward: 234.00 at episode 66
running reward: 201.00 at episode 67
running reward: 238.00 at episode 68
running reward: 249.00 at episode 69
running reward: 244.00 at episode 70
running reward: 237.00 at episode 71
running reward: 236.00 at episode 72
running reward: 207.00 at episode 73
running reward: 212.00 at episode 74
running reward: 187.00 at episode 75
running reward: 226.00 at episode 76
running reward: 203.00 at episode 77
running reward: 197.00 at episode 78
running reward: 224.00 at episode 79
running reward: 199.00 at episode 80
running reward: 214.00 at episode 81
running reward: 227.00 at episode 82
running reward: 189.00 at episode 83
running reward: 204.00 at episode 84
running reward: 208.00 at episode 85
running reward: 223.00 at episode 86
running reward: 196.00 at episode 87
running reward: 228.00 at episode 88
r